<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#连接-ES" data-toc-modified-id="连接-ES-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>连接 ES</a></span></li><li><span><a href="#插入数据" data-toc-modified-id="插入数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>插入数据</a></span></li><li><span><a href="#批量操作" data-toc-modified-id="批量操作-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>批量操作</a></span></li><li><span><a href="#查询数据" data-toc-modified-id="查询数据-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>查询数据</a></span></li><li><span><a href="#全文搜索" data-toc-modified-id="全文搜索-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>全文搜索</a></span></li><li><span><a href="#搜索词组" data-toc-modified-id="搜索词组-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>搜索词组</a></span></li><li><span><a href="#实战" data-toc-modified-id="实战-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>实战</a></span><ul class="toc-item"><li><span><a href="#读取-json-文件，导入数据" data-toc-modified-id="读取-json-文件，导入数据-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>读取 json 文件，导入数据</a></span></li><li><span><a href="#调试函数" data-toc-modified-id="调试函数-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>调试函数</a></span></li><li><span><a href="#写入数据库" data-toc-modified-id="写入数据库-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>写入数据库</a></span></li><li><span><a href="#验证写入的数据" data-toc-modified-id="验证写入的数据-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>验证写入的数据</a></span></li></ul></li><li><span><a href="#尝试解析输入" data-toc-modified-id="尝试解析输入-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>尝试解析输入</a></span></li><li><span><a href="#尝试添加特殊符号" data-toc-modified-id="尝试添加特殊符号-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>尝试添加特殊符号</a></span></li><li><span><a href="#查询及变更设置" data-toc-modified-id="查询及变更设置-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>查询及变更设置</a></span></li></ul></div>

参考资料

* [es python api文档](http://elasticsearch-py.readthedocs.io/en/latest/)
* [Elasticsearch: 权威指南](https://www.elastic.co/guide/cn/elasticsearch/guide/current/index.html)

## 连接 ES

In [244]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from datetime import datetime
import pprint

In [ ]:
es = Elasticsearch()

## 插入数据

In [24]:
doc = {
    'author': 'kimchy',
    'text': 'This is a brown fox',
    'timestamp': datetime.now(),
}
es.index(index="test-index", doc_type='tweet', id=1, body=doc)

{'_id': '1',
 '_index': 'test-index',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'tweet',
 '_version': 3,
 'result': 'updated'}

## 批量操作

In [25]:
doc2 = {
    'author': 'jiexiao',
    'text': 'This is a brown dog',
    'timestamp': datetime.now(),
}
doc3 = {
    'author': 'tangting',
    'text': 'This dog is really brown',
    'timestamp': datetime.now(),
}
doc4 = {
    'author': 'tangting',
    'text': 'The dog is brown but this document is very very long',
    'timestamp': datetime.now(),
}
doc5 = {
    'author': 'tangting',
    'text': 'There is also a white cat',
    'timestamp': datetime.now(),
}
doc6 = {
    'author': 'tangting',
    'text': 'The quick brown fox jumps over the lazy dog',
    'timestamp': datetime.now(),
}
actions = (
    { "_index": "test-index", "_type": "tweet", "_id": 2, "_source": doc2},
    { "_index": "test-index", "_type": "tweet", "_id": 3, "_source": doc3},
    { "_index": "test-index", "_type": "tweet", "_id": 4, "_source": doc4},
    { "_index": "test-index", "_type": "tweet", "_id": 5, "_source": doc5},
    { "_index": "test-index", "_type": "tweet", "_id": 6, "_source": doc6}
)
helpers.bulk(es, actions)  

(5, [])

## 查询数据

根据 ID 单条查询

In [8]:
es.get(index="test-index", doc_type='tweet', id=1)

{'_id': '1',
 '_index': 'test-index',
 '_source': {'author': 'kimchy',
  'text': 'Elasticsearch: cool. bonsai cool.',
  'timestamp': '2018-04-17T21:42:47.345907'},
 '_type': 'tweet',
 '_version': 2,
 'found': True}

In [27]:
es.indices.refresh(index="test-index")

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

查询所有条目

In [28]:
es.search(index="test-index", body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '5',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'tangting',
     'text': 'There is also a white cat',
     'timestamp': '2018-04-18T10:55:35.734267'},
    '_type': 'tweet'},
   {'_id': '2',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'jiexiao',
     'text': 'This is a brown dog',
     'timestamp': '2018-04-18T10:55:35.733938'},
    '_type': 'tweet'},
   {'_id': '4',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'tangting',
     'text': 'The dog is brown but this document is very very long',
     'timestamp': '2018-04-18T10:55:35.734050'},
    '_type': 'tweet'},
   {'_id': '6',
    '_index': 'test-index',
    '_score': 1.0,
    '_source': {'author': 'tangting',
     'text': 'The quick brown fox jumps over the lazy dog',
     'timestamp': '2018-04-18T10:55:35.734326'},
    '_type': 'tweet'},
   {'_id': '1',


## 全文搜索

In [29]:
es.search(index="test-index", body={"query": {"match": {"text": "quick brown dog tangting"}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '6',
    '_index': 'test-index',
    '_score': 1.2083462,
    '_source': {'author': 'tangting',
     'text': 'The quick brown fox jumps over the lazy dog',
     'timestamp': '2018-04-18T10:55:35.734326'},
    '_type': 'tweet'},
   {'_id': '3',
    '_index': 'test-index',
    '_score': 0.5753642,
    '_source': {'author': 'tangting',
     'text': 'This dog is really brown',
     'timestamp': '2018-04-18T10:55:35.733999'},
    '_type': 'tweet'},
   {'_id': '2',
    '_index': 'test-index',
    '_score': 0.31931418,
    '_source': {'author': 'jiexiao',
     'text': 'This is a brown dog',
     'timestamp': '2018-04-18T10:55:35.733938'},
    '_type': 'tweet'},
   {'_id': '1',
    '_index': 'test-index',
    '_score': 0.2876821,
    '_source': {'author': 'kimchy',
     'text': 'This is a brown fox',
     'timestamp': '2018-04-18T10:55:33.228343'},
    '_type': 'tweet'},
   {'_id': '4',
    '_index

## 搜索词组

In [30]:
es.search(index="test-index", body={"query": {"match_phrase": {"text": {"query": "brown  dog", "slop": 3}}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2',
    '_index': 'test-index',
    '_score': 0.31931418,
    '_source': {'author': 'jiexiao',
     'text': 'This is a brown dog',
     'timestamp': '2018-04-18T10:55:35.733938'},
    '_type': 'tweet'},
   {'_id': '4',
    '_index': 'test-index',
    '_score': 0.08451354,
    '_source': {'author': 'tangting',
     'text': 'The dog is brown but this document is very very long',
     'timestamp': '2018-04-18T10:55:35.734050'},
    '_type': 'tweet'}],
  'max_score': 0.31931418,
  'total': 2},
 'timed_out': False,
 'took': 3}

## 实战

### 读取 json 文件，导入数据

In [35]:
import pandas as pd

In [ ]:
df = pd.read_csv("/tmp/API_RECOM/aw_cmd.csv")

### 调试函数

In [38]:
df.columns

/root/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12,15,16,17,18,19,22,23,24,25,26,29,30,32,33,34,37,38,40,42,43,44,45,47,48,49,50,52,53,54,55,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['_id', 'id', 'id_label', 'file_name', 'file_path', 'type', 'belong_gem',
       'belong_feature', 'cmd_version.0', 'cmd_version.1', 'cmd_desc.1_5_0_0',
       'cmd_desc.1_5_1_0', 'cmd_desc.1_3_0_0', 'cmd_desc.8_1_1_0',
       'cmd_desc.8_2_0_0', 'cmd_desc.8_8_1_0', 'cmd_desc.1_18_0_0',
       'cmd_view.1_5_0_0', 'cmd_view.1_5_1_0', 'cmd_view.1_3_0_0',
       'cmd_view.8_1_1_0', 'cmd_view.8_2_0_0', 'cmd_view.8_8_1_0',
       'cmd_view.1_18_0_0', 'cmd_format.1_5_0_0', 'cmd_format.1_5_1_0',
       'cmd_format.1_3_0_0', 'cmd_format.8_1_1_0', 'cmd_format.8_2_0_0',
       'cmd_format.8_8_1_0', 'cmd_format.1_18_0_0', 'cmd_format_t',
       'cmd_source_code.1_5_0_0', 'cmd_source_code.1_5_1_0',
       'cmd_source_code.1_3_0_0', 'cmd_source_code.8_1_1_0',
       'cmd_source_code.8_2_0_0', 'cmd_source_code.8_8_1_0',
       'cmd_source_code.1_18_0_0', 'ret_version', 'ret_desc.8_8_1_0',
       'ret_desc.8_2_0_0', 'ret_desc.1_3_0_0', 'ret_desc.1_5_1_0',
       'ret_desc.1_5_0_0', 'ret_type.8_

In [66]:
df.head(1)

,_id,id,id_label,file_name,file_path,type,belong_gem,belong_feature,cmd_version.0,cmd_version.1,...,rpc_source_code,rpl_version,rpl_desc,rpl_xpath,rpl_format,rpl_source_code,created_by,created_date,last_updated_by,last_updated_date
0,5959a1722d0d8963d145256e,a5c1f9e2-9358-4a29-b419-a42436777586,cmd_display_dcn_encrypt_channel,CmdDcn.rb,Testlib-Vrp-Igp\\Dcn\\Igp\\CmdDcn.rb,NaN,ab47e47e-94ed-432f-b384-a30b1bcc40aa,e52e1502-cf82-4009-8275-fd94e3590179,8_15_0_0,NaN,...,{ },[ ],{ },{ },{ },{ },uuid~dzAwMzAzMTQ3,2017-07-03 09:44:18,uuid~dzAwMzAzMTQ3,2017-07-03 09:44:18


In [65]:
for idx, row in df.iterrows():
    type(idx)
    type(row)
    type(row['cmd_format_t'])
    row['cmd_format_t']
    break

int

pandas.core.series.Series

str

"'display dcn encrypt channel [ne_id %neid%]'\\r\\n \\n"

In [80]:
[(i, x) for i, x in enumerate(df['cmd_format_t'].isnull()) if x]

[(1310, True),
 (1311, True),
 (1312, True),
 (1313, True),
 (1327, True),
 (1365, True),
 (1368, True),
 (1369, True),
 (1370, True),
 (1377, True),
 (1378, True),
 (1379, True),
 (1380, True),
 (1381, True),
 (1382, True),
 (1383, True),
 (1384, True),
 (1385, True),
 (1387, True),
 (1400, True),
 (1425, True),
 (1435, True),
 (1436, True),
 (1437, True),
 (1438, True),
 (1439, True),
 (1440, True),
 (1441, True),
 (1442, True),
 (1676, True),
 (1677, True),
 (1678, True),
 (1679, True),
 (1728, True),
 (1729, True),
 (1730, True),
 (1854, True),
 (1855, True),
 (1858, True),
 (1859, True),
 (1885, True),
 (1888, True),
 (1889, True),
 (1890, True),
 (1891, True),
 (1892, True),
 (1893, True),
 (1894, True),
 (1896, True),
 (1899, True),
 (1900, True),
 (1901, True),
 (1902, True),
 (1903, True),
 (1904, True),
 (2239, True),
 (2240, True),
 (2241, True),
 (2263, True),
 (2332, True),
 (2333, True),
 (2334, True),
 (2339, True),
 (2583, True),
 (3428, True),
 (4193, True),
 (4226, Tr

In [86]:
tmp = df.iloc[5127]
tmp.to_dict()

{'_id': '5a0e44e02d0d892f7161ec2c',
 'belong_feature': '00e3a6eb-da23-45c4-881e-f3c668a801a3',
 'belong_gem': '583c29f0-c815-44e6-8c01-9b8928797c8d',
 'cmd_desc.1_18_0_0': nan,
 'cmd_desc.1_3_0_0': nan,
 'cmd_desc.1_5_0_0': nan,
 'cmd_desc.1_5_1_0': nan,
 'cmd_desc.8_1_1_0': nan,
 'cmd_desc.8_2_0_0': nan,
 'cmd_desc.8_8_1_0': nan,
 'cmd_format.1_18_0_0': nan,
 'cmd_format.1_3_0_0': nan,
 'cmd_format.1_5_0_0': nan,
 'cmd_format.1_5_1_0': nan,
 'cmd_format.8_1_1_0': nan,
 'cmd_format.8_2_0_0': nan,
 'cmd_format.8_8_1_0': nan,
 'cmd_format_t': nan,
 'cmd_source_code.1_18_0_0': nan,
 'cmd_source_code.1_3_0_0': nan,
 'cmd_source_code.1_5_0_0': nan,
 'cmd_source_code.1_5_1_0': nan,
 'cmd_source_code.8_1_1_0': nan,
 'cmd_source_code.8_2_0_0': nan,
 'cmd_source_code.8_8_1_0': nan,
 'cmd_version.0': nan,
 'cmd_version.1': nan,
 'cmd_view.1_18_0_0': nan,
 'cmd_view.1_3_0_0': nan,
 'cmd_view.1_5_0_0': nan,
 'cmd_view.1_5_1_0': nan,
 'cmd_view.8_1_1_0': nan,
 'cmd_view.8_2_0_0': nan,
 'cmd_view.8_

In [114]:
df.loc[2, 'cmd_format_t']

"'ipv6 icmp [ %icmp_mode% ] source-address'\\r\\n \\n"

In [78]:
len([x for x in df['id_label'].isnull() if x])

0

### 写入数据库

In [92]:
def decode_nginx_log(_df):
    for idx, row in _df.iterrows():
        # Filter out the below from each log line
        id_label = row['id_label']
        cmd_format_t = str(row['cmd_format_t'])

        # Index for elasticsearch. Typically timestamp.
        # idx = ...

        es_fields_keys = ('id_label', 'cmd_format_t')
        es_fields_vals = (id_label, cmd_format_t)

        # We return a dict holding values from each line
        es_nginx_d = dict(zip(es_fields_keys, es_fields_vals))

        # Return the row on each iteration
        yield idx, es_nginx_d   # <- Note the usage of 'yield'

def es_add_bulk(_df):
    es = Elasticsearch(hosts = [{'host': 'localhost', 'port': 9200}])

    # NOTE the (...) round brackets. This is for a generator.
    k = ({
            "_index": "nginx",
            "_type" : "logs",
            "_id"   : idx,
            "_source": es_nginx_d,
         } for idx, es_nginx_d in decode_nginx_log(_df))

    helpers.bulk(es, k)

# Now, just run it.
es_add_bulk(df)

### 验证写入的数据

In [93]:
es.search(index="nginx", body={"query": {"match_all": {}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '8002',
    '_index': 'nginx',
    '_score': 1.0,
    '_source': {'cmd_format_t': "'display meap interface brief'\\r\\n \\n",
     'id_label': 'cmd_meap_get_intf_brief'},
    '_type': 'logs'},
   {'_id': '8005',
    '_index': 'nginx',
    '_score': 1.0,
    '_source': {'cmd_format_t': "'display meap statistics'\\r\\n \\n",
     'id_label': 'cmd_meap_get_statistics'},
    '_type': 'logs'},
   {'_id': '8006',
    '_index': 'nginx',
    '_score': 1.0,
    '_source': {'cmd_format_t': "'meap'\\r\\n \\n",
     'id_label': 'cmd_meap_set'},
    '_type': 'logs'},
   {'_id': '8012',
    '_index': 'nginx',
    '_score': 1.0,
    '_source': {'cmd_format_t': "'sub-authorize-server'\\r\\n \\n",
     'id_label': 'cmd_meap_set_sub_authorize_server'},
    '_type': 'logs'},
   {'_id': '8018',
    '_index': 'nginx',
    '_score': 1.0,
    '_source': {'cmd_format_t': "'meap shutdown'\\r\\n \\n",
     'id_label

In [115]:
es.indices.analyze(index="nginx", body={"text": "mpls-te"})

{'tokens': [{'end_offset': 4,
   'position': 0,
   'start_offset': 0,
   'token': 'mpls',
   'type': '<ALPHANUM>'},
  {'end_offset': 7,
   'position': 1,
   'start_offset': 5,
   'token': 'te',
   'type': '<ALPHANUM>'}]}

In [101]:
es.search(index="nginx", body={"query": {"match_phrase": {"cmd_format_t": {"query": "undo mpls te", "slop": 30}}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '8310',
    '_index': 'nginx',
    '_score': 12.946407,
    '_source': {'cmd_format_t': "'undo mpls-te'\\r\\n \\n",
     'id_label': 'cmd_ospf_area_te_disable'},
    '_type': 'logs'},
   {'_id': '32670',
    '_index': 'nginx',
    '_score': 12.946407,
    '_source': {'cmd_format_t': "'undo mpls-te'\\r\\n \\n",
     'id_label': 'cmd_unset_inclusive_root_mplste_view'},
    '_type': 'logs'},
   {'_id': '33219',
    '_index': 'nginx',
    '_score': 12.946407,
    '_source': {'cmd_format_t': "'undo mpls-te'\\r\\n \\n",
     'id_label': 'cmd_unset_inclusive_root_mplste_view'},
    '_type': 'logs'},
   {'_id': '30708',
    '_index': 'nginx',
    '_score': 12.927313,
    '_source': {'cmd_format_t': "'undo mpls te'\\r\\n \\n",
     'id_label': 'cmd_bgp_mvpn_ipmsi_tunnel_unset_te'},
    '_type': 'logs'},
   {'_id': '7058',
    '_index': 'nginx',
    '_score': 12.763281,
    '_source': {'cmd_format_t'

## 尝试解析输入

In [116]:
es.indices.analyze(index="nginx", body={"text": "mpls-te"})

{'tokens': [{'end_offset': 4,
   'position': 0,
   'start_offset': 0,
   'token': 'mpls',
   'type': '<ALPHANUM>'},
  {'end_offset': 7,
   'position': 1,
   'start_offset': 5,
   'token': 'te',
   'type': '<ALPHANUM>'}]}

## 尝试添加特殊符号

In [214]:
test_hyphen = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "standard",
          "char_filter": [
            "my_char_filter"
          ]
        }
      },
      "char_filter": {
        "my_char_filter": {
          "type": "mapping",
          "mappings": [
            "- => _hyphen_",
          ]
        }
      }
    }
  }
}

In [ ]:
map_test_hyphen = {'properties': {
    'text': {
        "search_analyzer": "my_analyzer",
        "analyzer": "my_analyzer",
        'type': 'text'}}}
es.indices.put_mapping(index="test_hyphen", body=map_test_hyphen, doc_type='tweet')

In [268]:
es.indices.delete(index="test_hyphen")
es.indices.create(index="test_hyphen", body=test_hyphen)

{'acknowledged': True}

{'acknowledged': True, 'index': 'test_hyphen', 'shards_acknowledged': True}

In [123]:
es.indices.analyze(index="test_hyphen", body={"analyzer": "my_analyzer", "text": "mpls-te"})

{'tokens': [{'end_offset': 7,
   'position': 0,
   'start_offset': 0,
   'token': 'mpls_hyphen_te',
   'type': '<ALPHANUM>'}]}

## 查询及变更设置

In [280]:
pprint.pprint(es.indices.get_settings(index="test_hyphen"))

{'test_hyphen': {'settings': {'index': {'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['my_char_filter'],
                                                                                  'tokenizer': 'standard'}},
                                                     'char_filter': {'my_char_filter': {'mappings': ['[ '
                                                                                                     '=> '
                                                                                                     '_l_square_brackets_',
                                                                                                     '] '
                                                                                                     '=> '
                                                                                                     '_r_square_brackets_'],
                                                                                        'type': 'ma

In [271]:
pprint.pprint(es.indices.get_mapping(index="test_hyphen"))

{'test_hyphen': {'mappings': {'tweet': {'properties': {'text': {'analyzer': 'my_analyzer',
                                                                'type': 'text'}}}}}}


In [270]:
map_test_hyphen = {'properties': {
    'text': {
        "search_analyzer": "my_analyzer",
        "analyzer": "my_analyzer",
        'type': 'text'}}}
es.indices.put_mapping(index="test_hyphen", body=map_test_hyphen, doc_type='tweet')

{'acknowledged': True}

In [284]:
doc = {
    'text': 'This is-a [brown fox]',
}
doc2 = {
    'text': 'This is a brown fox',
}
doc3 = {
    'text': 'This is_a brown fox',
}
doc4 = {
    'text': 'This is-a [brown fox]',
}
es.index(index="test_hyphen", doc_type='tweet', id=1, body=doc)
es.index(index="test_hyphen", doc_type='tweet', id=2, body=doc2)
es.index(index="test_hyphen", doc_type='tweet', id=3, body=doc3)
es.index(index="test_hyphen", doc_type='tweet', id=4, body=doc4)

{'_id': '1',
 '_index': 'test_hyphen',
 '_primary_term': 3,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'tweet',
 '_version': 3,
 'result': 'updated'}

{'_id': '2',
 '_index': 'test_hyphen',
 '_primary_term': 3,
 '_seq_no': 4,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'tweet',
 '_version': 3,
 'result': 'updated'}

{'_id': '3',
 '_index': 'test_hyphen',
 '_primary_term': 3,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'tweet',
 '_version': 3,
 'result': 'updated'}

{'_id': '4',
 '_index': 'test_hyphen',
 '_primary_term': 3,
 '_seq_no': 5,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'tweet',
 '_version': 3,
 'result': 'updated'}

In [275]:
test_hyphen_2 = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "tokenizer": "standard",
          "char_filter": [
            "my_char_filter"
          ]
        }
      },
      "char_filter": {
        "my_char_filter": {
          "type": "mapping",
          "mappings": [
            "[ => _l_square_brackets_",
            "] => _r_square_brackets_"
          ]
        }
      }
    }
  }
}

变更配置

In [286]:
es.indices.close(index="test_hyphen")
es.indices.put_settings(index="test_hyphen", body=test_hyphen_2)
es.indices.open(index="test_hyphen")
es.indices.get_settings(index="test_hyphen")

{'acknowledged': True}

{'acknowledged': True}

{'acknowledged': True, 'shards_acknowledged': True}

{'test_hyphen': {'settings': {'index': {'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['my_char_filter'],
       'tokenizer': 'standard'}},
     'char_filter': {'my_char_filter': {'mappings': ['[ => _l_square_brackets_',
        '] => _r_square_brackets_'],
       'type': 'mapping'}}},
    'creation_date': '1524046326806',
    'number_of_replicas': '1',
    'number_of_shards': '5',
    'provided_name': 'test_hyphen',
    'uuid': 'cvIBpXjWTfSX6PiLXzNq9Q',
    'version': {'created': '6020399'}}}}}

In [289]:
es.search(index="test_hyphen", body={"query": {"match": {"text": "is"}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2',
    '_index': 'test_hyphen',
    '_score': 0.66301036,
    '_source': {'text': 'This is a brown fox'},
    '_type': 'tweet'}],
  'max_score': 0.66301036,
  'total': 1},
 'timed_out': False,
 'took': 2}

In [274]:
es.search(index="test_hyphen", body={"query": {"match": {"text": "is-a"}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'test_hyphen',
    '_score': 0.7261542,
    '_source': {'text': 'This is-a [brown fox]'},
    '_type': 'tweet'},
   {'_id': '1',
    '_index': 'test_hyphen',
    '_score': 0.2876821,
    '_source': {'text': 'This is-a [brown fox]'},
    '_type': 'tweet'}],
  'max_score': 0.7261542,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [160]:
es.search(index="test_hyphen", body={"query": {"match": {"text": "[brown"}}})

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '2',
    '_index': 'test_hyphen',
    '_score': 0.2876821,
    '_source': {'text': 'This is_a brown fox'},
    '_type': 'tweet'},
   {'_id': '1',
    '_index': 'test_hyphen',
    '_score': 0.2876821,
    '_source': {'text': 'This is_a [brown fox]'},
    '_type': 'tweet'}],
  'max_score': 0.2876821,
  'total': 2},
 'timed_out': False,
 'took': 3}

In [288]:
es.indices.refresh(index="test_hyphen")
es.indices.flush(index="test_hyphen")
es.indices.flush_synced(index="test_hyphen")

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

{'_shards': {'failed': 0, 'successful': 5, 'total': 10},
 'test_hyphen': {'failed': 0, 'successful': 5, 'total': 10}}

In [227]:
es.indices.analyze(index="test_hyphen", body={"text": "mpls-te", "analyzer": "my_analyzer"})

{'tokens': [{'end_offset': 7,
   'position': 0,
   'start_offset': 0,
   'token': 'mpls_hyphen_te',
   'type': '<ALPHANUM>'}]}